In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2021-07'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

In [3]:
res.text

'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=57423"><img src="https://image.cine21.com/resize/cine21/person/2019/0410/18_00_45__5cadb0bd79e87[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=57423">김윤석(1편)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">흥행지수</span><strong>143,866</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- 영화포스터는 최대 5개까지만 -->\r\n\t\t\t\t\t<ul class="mov_list">\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t<li>\r\n\t\t\t\t\t\t\t\t<a href="/movie/info/?movie_id=57275">\r\n\t\t\t\t\t\t\t\t\t<img src="https://image.cine21.com/resize/cine21/poster/2021/0728/15_15_08__6100f5eca274b[X85,120].jpg" alt="" class="thumb"  target="_blank" />\r\n\t\t\t\

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))    

김윤석
조인성
허준호
김소진
정만식
조우진
구교환


In [6]:
ranking = soup.select('li.people_li span.grade')
for rank in ranking:
       print(rank.text)
        
#오류 발생 : ranking.text
ranking[1].text

1
2
3
4
5
6
7


'2'

In [7]:
num = soup.select("li.people_li ul.num_info li strong")
for num in num:
    print(num.text)

143,866
125,883
107,901
89,927
71,933
66,136
53,950


In [8]:
movie = soup.select("li.people_li ul.mov_list")

#print(movie[2].text)
#print(movie[5].text)

movie_ = movie[2].text.strip().replace("\n\n\n\n\n\n", ",")
movie_ = movie_.split(",")
movie_

['모가디슈', '국가부도의 날']

최종 코드

In [9]:
actors = soup.select("li.people_li div.name")   
num = soup.select("li.people_li ul.num_info li strong") #흥행성적
movie = soup.select("li.people_li ul.mov_list")  #최근 참여 영화
rank = soup.select("li.people_li span.grade") #랭킹

actors_info_list = list()

for i in range(1,11):
    
    cine21_url = 'http://www.cine21.com/rank/person/content'
    post_data = dict()
    post_data['section'] = 'actor'
    post_data['period_start'] = '2021-07'
    post_data['gender'] = 'all'
    post_data['page'] = i

    res = requests.post(cine21_url, data=post_data)

    soup = BeautifulSoup(res.text, 'html.parser')
    
    for index,actor in enumerate(actors):
        
        actor_info_dict = dict()
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_info_dict['이름'] = actor_name
        actor_info_dict['흥행지수'] = num[index].text
        actor_info_dict['랭킹'] = rank[index].text
        movie_ = movie[index].text.strip().replace("\n\n\n\n\n\n", " ")
        movie_ = movie_.split(",")
        actor_info_dict["출연영화"] = movie_
        
        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')
    
        for actor_detail in actor_details:
            actor_detail_key = actor_detail.select_one('span.tit').text
            actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
            actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
            actor_info_dict[actor_detail_key] = actor_detail_value
        
        actors_info_list.append(actor_info_dict)

actors_info_list        

[{'이름': '김윤석',
  '흥행지수': '143,866',
  '랭킹': '1',
  '출연영화': ['모가디슈'],
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'이름': '조인성',
  '흥행지수': '125,883',
  '랭킹': '2',
  '출연영화': ['모가디슈'],
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'이름': '허준호',
  '흥행지수': '107,901',
  '랭킹': '3',
  '출연영화': ['모가디슈 국가부도의 날'],
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'이름': '김소진',
  '흥행지수': '89,927',
  '랭킹': '4',
  '출연영화': ['모가디슈 아이 캔 스피크'],
  '직업': '배우',
  '성별': '여'},
 {'이름': '정만식',
  '흥행지수': '71,933',
  '랭킹': '5',
  '출연영화': ['모가디슈'],
  '직업': '배우',
  '성별': '남'},
 {'이름': '조우진',
  '흥행지수': '66,136',
  '랭킹': '6',
  '출연영화': ['발신제한 